In [1]:
import sys
sys.path.append('C:\\projects\\streamlit_langchain_doc_answering')

import os
from langchain.document_loaders import PyPDFLoader

In [2]:
pdf_loader = PyPDFLoader("../data/raw/231231_wh40k_core_rules_10th_edition.pdf")
documents = pdf_loader.load()

In [3]:
documents_small = documents[:3]
documents_small

[Document(page_content='Core Rules\n‘ W e are beset on all sides by vile predatory aliens \nand sedition gnaws at us from within; in this dark hour \nthe best we can do is look to our wargear and pray to our gods.’\n- Skolak a’Trellar IV, Imperial Commander', metadata={'source': '../data/raw/231231_wh40k_core_rules_10th_edition.pdf', 'page': 0}),
 Document(page_content='', metadata={'source': '../data/raw/231231_wh40k_core_rules_10th_edition.pdf', 'page': 1}),
 Document(page_content='Welcome to the Warhammer 40,000 \nCore Rules! The following pages contain \neverything you need to know in order to wage glorious battle across the war-torn galaxy of the 41st Millennium.\nWarhammer 40,000 is a tabletop war \ngame in which players command armies of Citadel miniatures and attempt to defeat their opponent through a mixture \nof skill, tactics and luck. Storytelling is \nat the core of Warhammer 40,000, with the rules designed to bring to life the epic conflicts between the forces of Mankind,

In [4]:
# Set OpenAI API key
import json
with open('../credentials.json') as f:
    data = json.load(f)
    openai_api_key = data['OPEN_AI_API_KEY']

In [5]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

# we are specifying that OpenAI is the LLM that we want to use in our chain

chain = load_qa_chain(llm=OpenAI(openai_api_key=openai_api_key), verbose=True)
query = 'What is the theory of relativity?'
response = chain.run(input_documents=documents_small, question=query)
print(response) 



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Core Rules
‘ W e are beset on all sides by vile predatory aliens 
and sedition gnaws at us from within; in this dark hour 
the best we can do is look to our wargear and pray to our gods.’
- Skolak a’Trellar IV, Imperial Commander



Welcome to the Warhammer 40,000 
Core Rules! The following pages contain 
everything you need to know in order to wage glorious battle across the war-torn galaxy of the 41st Millennium.
Warhammer 40,000 is a tabletop war 
game in which players command armies of Citadel miniatures and attempt to defeat their opponent through a mixture 
of skill, tactics and luck. Storytelling is 
at the core of Warhammer 40,000, with the rules designed to bring to life the epic conflicts between the forces of 

In [6]:
from single_long_doc import load_data

db = load_data(openai_api_key)

# instantiate the database retriever
retriever = db.as_retriever()

In [13]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(openai_api_key=openai_api_key),
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
    chain_type_kwargs={
        "verbose": True,
    }
)

# we can now execute queries against our Q&A chain
result = qa_chain({'query': 'What is unit coherency?'})
print(result['result'])



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

CORE RULES | CORE CONCEPTSUNIT COHERENCY
A unit that contains more than one model must be set 
up and end any kind of move as a single group, with all of its models within 2" horizontally and 5" vertically of at least one other model from that unit. While a unit has seven or more models, all of its models must instead be set up and end any kind of move within 2" horizontally and 5" vertically of at least two other models from that unit. This is called Unit Coherency. 
If for any reason a model cannot be set up in Unit 
Coherency, that model is destroyed. If a unit cannot end a move in Unit Coherency, it cannot make that move and its models are returned to their previous positions.
At

In [12]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:
